In [1]:
seed = 161
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Composicion de pipelines
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Regresion lineal
from sklearn.linear_model import LinearRegression

# Importar/ Exportar modelos
from joblib import dump, load

# Metricas
from sklearn.metrics import mean_squared_error as mse

# q-q plots
import scipy.stats as stats

import json

In [2]:
df_conflicto = pd.read_excel("Conflicto Armado.xlsx")
df_conflicto.shape

(314119, 13)

In [3]:
df_conflicto.sample(5)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
281054,47.0,Magdalena,47170.0,Chibolo,Conflicto armado y seguridad ciudadana,Violencia de pareja según sexo de la víctima,Violencia de pareja - Total de casos,"6,00",NaN,2014.0,12.0,Instituto Nacional de Medicina Legal y Ciencia...,Personas
151664,52.0,Nariño,52258.0,El Tablón de Gómez,Conflicto armado y seguridad ciudadana,Seguridad Ciudadana,Tasa de violencia intrafamiliar (x cada 100.00...,"7,22",NaN,2006.0,12.0,DNP a partir de información de la Fiscalía Gen...,Casos por cada 100.000 habitantes
156242,54.0,Norte de Santander,54377.0,Labateca,Conflicto armado y seguridad ciudadana,Seguridad Ciudadana,Tasa de violencia intrafamiliar (x cada 100.00...,"0,00",NaN,2010.0,12.0,DNP a partir de información de la Fiscalía Gen...,Casos por cada 100.000 habitantes
41762,68.0,Santander,68684.0,San José de Miranda,Conflicto armado y seguridad ciudadana,Conflicto armado,Número acumulado de personas desplazadas recib...,"30,00",1984 - 2017,2017.0,12.0,Unidad para la Atención y Reparación Integral ...,Personas
156261,54.0,Norte de Santander,54000.0,Norte de Santander,Conflicto armado y seguridad ciudadana,Seguridad Ciudadana,Tasa de violencia intrafamiliar (x cada 100.00...,"15,26",NaN,2010.0,12.0,DNP a partir de información de la Fiscalía Gen...,Casos por cada 100.000 habitantes


In [4]:
df_conflicto.dtypes

Código Departamento    float64
Departamento            object
Código Entidad         float64
Entidad                 object
Dimensión               object
Subcategoría            object
Indicador               object
Dato Numérico           object
Dato Cualitativo        object
Año                    float64
Mes                    float64
Fuente                  object
Unidad de Medida        object
dtype: object

In [5]:
df_conflicto.info()
#Dato numerico tiene cerca de 60k nulos
#Dato cualitativo tiene muchos nulos, revisar y si no es rescatable, dropear

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314119 entries, 0 to 314118
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Código Departamento  314118 non-null  float64
 1   Departamento         314118 non-null  object 
 2   Código Entidad       314118 non-null  float64
 3   Entidad              314118 non-null  object 
 4   Dimensión            314118 non-null  object 
 5   Subcategoría         314118 non-null  object 
 6   Indicador            314119 non-null  object 
 7   Dato Numérico        259158 non-null  object 
 8   Dato Cualitativo     4536 non-null    object 
 9   Año                  314118 non-null  float64
 10  Mes                  314118 non-null  float64
 11  Fuente               314119 non-null  object 
 12  Unidad de Medida     314118 non-null  object 
dtypes: float64(4), object(9)
memory usage: 31.2+ MB


In [6]:
df_conflicto.describe()

,Código Departamento,Código Entidad,Año,Mes
count,314118.000000,314118.000000,314118.000000,314118.0
mean,37.666667,38084.496473,2009.537906,12.0
std,26.018316,25990.452996,8.537646,0.0
min,1.000000,1001.000000,1984.000000,12.0
25%,15.000000,15667.000000,2006.000000,12.0
50%,25.000000,25806.000000,2014.000000,12.0
75%,66.000000,66000.000000,2015.000000,12.0
max,99.000000,99773.000000,2018.000000,12.0


In [7]:
pd.value_counts(df_conflicto['Departamento'])
#Que hago con las de colombia, los dropeo, son pocos

Antioquia                   34902
Boyacá                      34348
Cundinamarca                32409
Santander                   24376
Nariño                      18005
Tolima                      13296
Bolívar                     13019
Valle del Cauca             11911
Cauca                       11911
Norte de Santander          11357
Huila                       10526
Córdoba                      8587
Chocó                        8587
Magdalena                    8587
Meta                         8310
Caldas                       7756
Sucre                        7479
Cesar                        7202
Atlántico                    6648
Casanare                     5540
Caquetá                      4709
La Guajira                   4432
Risaralda                    4155
Putumayo                     3878
Quindío                      3601
Arauca                       2216
Guaviare                     1385
Vichada                      1385
Vaupés                       1108
Amazonas      

In [8]:
#Se quita Colombia
def quitarColombia(row):
    if row=="Colombia":
        return None
    return row

df_conflicto['Departamento'] = df_conflicto['Departamento'].apply(quitarColombia)
pd.value_counts(df_conflicto['Departamento'])

Antioquia                   34902
Boyacá                      34348
Cundinamarca                32409
Santander                   24376
Nariño                      18005
Tolima                      13296
Bolívar                     13019
Cauca                       11911
Valle del Cauca             11911
Norte de Santander          11357
Huila                       10526
Córdoba                      8587
Chocó                        8587
Magdalena                    8587
Meta                         8310
Caldas                       7756
Sucre                        7479
Cesar                        7202
Atlántico                    6648
Casanare                     5540
Caquetá                      4709
La Guajira                   4432
Risaralda                    4155
Putumayo                     3878
Quindío                      3601
Arauca                       2216
Guaviare                     1385
Vichada                      1385
Vaupés                       1108
Amazonas      

In [9]:
pd.value_counts(df_conflicto['Entidad'])
#nada relevante aqui

Buenavista    1108
La Unión      1108
Córdoba       1108
Nariño        1108
Bolívar       1108
              ... 
Padilla        277
Paéz           277
Patía          277
Piamonte       277
Cumaribo       277
Name: Entidad, Length: 1044, dtype: int64

In [10]:
pd.value_counts(df_conflicto['Dimensión'])

Conflicto armado y seguridad ciudadana    314118
Name: Dimensión, dtype: int64

In [11]:
pd.value_counts(df_conflicto['Subcategoría'])
#Los importantes son homicidios por agresor y sexo de la victima
#Presuntos delitos sexuales segun sexo de la victima
#Violencia de pareja
#Violencia de genero

Conflicto armado                                       113400
Homicidios por agresor y sexo de la víctima             95256
Seguridad Ciudadana                                     51030
Presuntos delitos sexuales según sexo de la víctima     20412
Violencia de pareja según sexo de la víctima            20412
Percepción                                               7938
Violencias de género                                     5670
Name: Subcategoría, dtype: int64

In [12]:
pd.value_counts(df_conflicto['Dato Numérico'])
#Cada dato numerico debe estar asociado a la unidad de medida asi que no tiene sentido mirar si hay outliers pero se deben transformar en numericos

0,00      128216
1,00       13347
2,00        6313
3,00        4128
4,00        3119
           ...  
200,12         1
407,55         1
160,45         1
61,43          1
176,19         1
Name: Dato Numérico, Length: 25029, dtype: int64

In [13]:
#Se arreglan los datos numericos
def arreglarNumerico(row):
    row=str(row).replace('.','')
    row = row.replace(',','.')
    return float(row)

df_conflicto['Dato Numérico'] = df_conflicto['Dato Numérico'].apply(arreglarNumerico)
pd.value_counts(df_conflicto['Dato Numérico'])

0.00      128216
1.00       13347
2.00        6313
3.00        4128
4.00        3119
           ...  
200.12         1
407.55         1
160.45         1
61.43          1
176.19         1
Name: Dato Numérico, Length: 25029, dtype: int64

In [14]:
pd.value_counts(df_conflicto['Dato Cualitativo'])
#Que representan los datos cualitativos?
#Se supone que es un intervalo de tiempo. Solo existen 2 intervalos y casi todos los
#demas registros son nulos, la infromacion se considera perdida

1984 - 2017    3402
1990-2017      1134
Name: Dato Cualitativo, dtype: int64

In [15]:
df_conflicto = df_conflicto.drop(['Dato Cualitativo'], axis=1)
#En caso de no poder recuperar los nulos de dato cuantitativo, se pueden dropear

In [16]:
pd.value_counts(df_conflicto['Año'])
#Los años tienen buen formato

2016.0    61236
2015.0    53298
2014.0    53298
2017.0    12474
2008.0     6804
2003.0     6804
2004.0     6804
2005.0     6804
2006.0     6804
2007.0     6804
2010.0     6804
2009.0     6804
2011.0     6804
2012.0     6804
2013.0     6804
2001.0     3402
2002.0     3402
2000.0     3402
1992.0     3402
1998.0     3402
1997.0     3402
1996.0     3402
1995.0     3402
1994.0     3402
1993.0     3402
1999.0     3402
1991.0     3402
1990.0     3402
1987.0     2268
1985.0     2268
1986.0     2268
1984.0     2268
1988.0     2268
1989.0     2268
2018.0     1134
Name: Año, dtype: int64

In [17]:
pd.value_counts(df_conflicto['Mes'])
#Todos los registros son en el ultimo mes, tiene sentido

12.0    314118
Name: Mes, dtype: int64

In [18]:
pd.value_counts(df_conflicto['Fuente'])

Instituto Nacional de Medicina Legal y Ciencias Forenses - INMLCF                                                                                                                                                                                                                                                                                                                                                                                                                                                      136080
Unidad para la Atención y Reparación Integral a las Víctimas                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [19]:
pd.value_counts(df_conflicto['Unidad de Medida'])

Personas                                                      222264
Casos por cada 100.000 habitantes                              57834
Porcentaje (el valor está multiplicado por 100)                 7938
Casos por cada 100.000 habitantes hombres                       6804
Casos por cada 100.000 habitantes mujeres                       6804
Casos de violencia hacia mujeres  por cada 100.000 mujeres      5670
Casos por cada 100.000 hombres                                  3402
Casos por cada 100.000 mujeres                                  3402
Name: Unidad de Medida, dtype: int64

In [20]:
df_conflicto=df_conflicto.dropna()

In [21]:
df_depymun = pd.read_csv("Departamentos_y_municipios_de_Colombia.csv")
df_depymun

,REGION,CÓDIGO DANE DEL DEPARTAMENTO,DEPARTAMENTO,CÓDIGO DANE DEL MUNICIPIO,MUNICIPIO
0,Región Eje Cafetero - Antioquia,5,Antioquia,5.001,Medellín
1,Región Eje Cafetero - Antioquia,5,Antioquia,5.002,Abejorral
2,Región Eje Cafetero - Antioquia,5,Antioquia,5.004,Abriaquí
3,Región Eje Cafetero - Antioquia,5,Antioquia,5.021,Alejandría
4,Región Eje Cafetero - Antioquia,5,Antioquia,5.030,Amagá
...,...,...,...,...,...
1118,Región Eje Cafetero - Antioquia,17,Caldas,17.444,Marquetalia
1119,Región Pacífico,52,Nariño,52.051,Arboleda
1120,Región Pacífico,76,Valle del Cauca,76.109,Buenaventura
1121,Región Caribe,47,Magdalena,47.189,Ciénaga


In [22]:
pd.value_counts(df_conflicto['Indicador'])

Número de personas secuestradas                                                                                     38522
Número de personas desplazadas                                                                                      38522
Número de personas víctimas de minas antipersona                                                                    31724
Tasa de homicidios (x cada 100.000 habitantes)                                                                      16978
Tasa de violencia intrafamiliar (x cada 100.000 habitantes)                                                         16934
Tasa de hurtos (x cada 100.000 habitantes)                                                                          14021
Tasa de violencias basadas en género                                                                                 5665
Tasa de presuntos delitos sexuales hacia mujeres (x cada 100.000 mujeres)                                            2794
Presuntos delitos sexual

In [23]:
municipios = {} 

for i, row in df_depymun.iterrows():
    municipio = row['MUNICIPIO'].lower().replace("í","i").replace("á","a").replace("é","e").replace("ó","o").replace("ú","u")
    departamento = row['DEPARTAMENTO'].lower().replace("í","i").replace("á","a").replace("é","e").replace("ó","o").replace("ú","u")
    municipios[municipio] = departamento

In [24]:
columns=['id','municipio', 'departamento'] #Este va a tener [id, municipio, departamento, columnasMineria, columnasSalud, columnasConflicto]
columns_pos = {'id':0,'municipio':1, 'departamento':2}

count = 3

columns.append('Violencia hacia la mujer 2014')
columns_pos['Violencia hacia la mujer 2014'] = count
count +=1 
columns.append('Violencia hacia la mujer 2015')
columns_pos['Violencia hacia la mujer 2015'] = count
count +=1 
columns.append('Violencia hacia la mujer 2016')
columns_pos['Violencia hacia la mujer 2016'] = count
count +=1 
columns.append('Violencia hacia la mujer 2017')
columns_pos['Violencia hacia la mujer 1999-2003'] = count
count +=1 
columns.append('Violencia hacia la mujer 2018')
columns_pos['Violencia hacia la mujer 2018'] = count
count +=1 

dfinal_mineria = pd.DataFrame(columns=columns)

In [25]:
valores_mineria = {}

for i, row in df_conflicto.iterrows():
    municipio = row['Entidad'].split("-")[0].lower().strip()
    departamento = row['Departamento'].lower()
    llave = municipio + "-" + departamento
    if llave not in valores_mineria:
      new_row = [llave, municipio, departamento]
      for i in range(0,len(columns)-2):
        new_row.append(0)
      valores_mineria[llave] = new_row
    if row["Indicador"]== "Homicidio de mujeres:  Total" or row["Indicador"]== "Violencia de pareja - Víctima mujer":
        pos = columns_pos['Violencia hacia la mujer 2014']
        if float(row["Año"])== float(2014):
            pos = columns_pos['Violencia hacia la mujer 2014']
        elif float(row["Año"])== float(2015):
            pos = columns_pos['Violencia hacia la mujer 2015']
        elif float(row["Año"])== float(2016):
            pos = columns_pos['Violencia hacia la mujer 2016']
        elif float(row["Año"])== float(2017):
            pos = columns_pos['Violencia hacia la mujer 2017']
        elif float(row["Año"])== float(2018):
            pos = columns_pos['Violencia hacia la mujer 2018']
        valor_viejo = valores_mineria[llave][pos]
        valor_a_sumar=row['Dato Numérico']
        valores_mineria[llave][pos] = valor_viejo + valor_a_sumar

print(columns)
print(valores_mineria)

['id', 'municipio', 'departamento', 'Violencia hacia la mujer 2014', 'Violencia hacia la mujer 2015', 'Violencia hacia la mujer 2016', 'Violencia hacia la mujer 2017', 'Violencia hacia la mujer 2018']
{'antioquia-antioquia': ['antioquia-antioquia', 'antioquia', 'antioquia', 179.0, 4207.0, 4411.0, 0, 0, 0], 'medellín-antioquia': ['medellín-antioquia', 'medellín', 'antioquia', 50.0, 2709.0, 2699.0, 0, 0, 0], 'abejorral-antioquia': ['abejorral-antioquia', 'abejorral', 'antioquia', 0.0, 5.0, 11.0, 0, 0, 0], 'abriaquí-antioquia': ['abriaquí-antioquia', 'abriaquí', 'antioquia', 0.0, 0, 0, 0, 0, 0], 'alejandría-antioquia': ['alejandría-antioquia', 'alejandría', 'antioquia', 0.0, 0.0, 0.0, 0, 0, 0], 'amagá-antioquia': ['amagá-antioquia', 'amagá', 'antioquia', 1.0, 1.0, 2.0, 0, 0, 0], 'amalfi-antioquia': ['amalfi-antioquia', 'amalfi', 'antioquia', 6.0, 2.0, 21.0, 0, 0, 0], 'andes-antioquia': ['andes-antioquia', 'andes', 'antioquia', 4.0, 56.0, 29.0, 0, 0, 0], 'angelópolis-antioquia': ['angelópo